# [CSC8101] Big Data Analytics - 2022 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd
from pyspark.sql.window import Window

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'L'
DATA_FORMAT = 'parquet'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)

Trips dataset loaded!
 ---
 Size: L
 Format: parquet
 Tables loaded: ['/FileStore/tables/taxi/tripdata_2020_21.parquet']
 Number of trips (dataset rows): 41,953,716

In [0]:
print_count(trips)

Row count: 132,396,785

In [0]:
# dataset schemas
trips.printSchema()

root
-- index: long (nullable = true)
-- VendorID: double (nullable = true)
-- tpep_pickup_datetime: string (nullable = true)
-- tpep_dropoff_datetime: string (nullable = true)
-- passenger_count: double (nullable = true)
-- trip_distance: double (nullable = true)
-- RatecodeID: double (nullable = true)
-- store_and_fwd_flag: string (nullable = true)
-- PULocationID: long (nullable = true)
-- DOLocationID: long (nullable = true)
-- payment_type: double (nullable = true)
-- fare_amount: double (nullable = true)
-- extra: double (nullable = true)
-- mta_tax: double (nullable = true)
-- tip_amount: double (nullable = true)
-- tolls_amount: double (nullable = true)
-- improvement_surcharge: double (nullable = true)
-- total_amount: double (nullable = true)
-- congestion_surcharge: double (nullable = true)
-- cab_type: string (nullable = true)
-- lpep_pickup_datetime: string (nullable = true)
-- lpep_dropoff_datetime: string (nullable = true)
-- ehail_fee: double (nullable = true)
-- trip_type: double (nullable = true)

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
151,239,1.5,1.0,9.95
239,246,2.6,1.0,16.3
236,236,0.0,3.0,5.8
193,193,0.0,5.0,7.55
193,193,0.0,5.0,55.55


In [0]:
zone_names.printSchema()

root
-- LocationID: long (nullable = true)
-- Borough: string (nullable = true)
-- Zone: string (nullable = true)
-- service_zone: string (nullable = true)

In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.438 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    # input: trips dataset
    df = df.selectExpr('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')
    df = df.filter(df.trip_distance != 0) 
    df = df.filter((df.passenger_count != 0) & (df.total_amount != 0))
    #display(df.take(5))
    return df

#### Task 1.1 - Explanation
In the above cell, I selected the necessary columns and overwrite the original dataframe with the dataframe with the selected columns. Next, I used the filter function twice to filter the values of trip distance equal to zero, the values of passenger count equal to zero and the values of total amount equal to zero. Then the final dataframe with filter values and column will be the return value of the function.

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
# display(trips_11.take(10))

Row count: 125,535,704

In [0]:
# Your solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    #trip_distance
    mediantrdist = df.agg(pf.percentile_approx('trip_distance', 0.5).alias('median'))
    #display(mediantrdist)
    for row in mediantrdist.collect():
        mediantrdistvalue = row['median']  
    df = df.withColumn("mediantrlist", df.trip_distance - mediantrdistvalue)
    df = df.withColumn("mediantrlistabs", pf.abs(df.mediantrlist))
    mediantrperc = df.agg(pf.percentile_approx('mediantrlistabs', 0.5).alias('median'))
    for row in mediantrperc.collect():
        mediantrpercvalue = 1.438 * row['median'] 
    df = df.withColumn("z_scoretripdist", df.mediantrlist / mediantrpercvalue)
    df = df.withColumn("z_scoretripdistabs", pf.abs(df.z_scoretripdist))
    df = df.filter(df.z_scoretripdistabs < 3.5)
    #total_amount
    mediantotam = df.agg(pf.percentile_approx('total_amount', 0.5).alias('median'))
    #display(mediantotam)
    for row in mediantotam.collect():
        mediantotamvalue = row['median']  
    df = df.withColumn("mediantotlist", df.trip_distance - mediantotamvalue)
    df = df.withColumn("mediantotlistabs", pf.abs(df.mediantotlist))
    mediantotperc = df.agg(pf.percentile_approx('mediantotlistabs', 0.5).alias('median'))
    for row in mediantotperc.collect():
        mediantotpercvalue = 1.438 * row['median'] 
    df = df.withColumn("z_scoretotdist", df.mediantotlist / mediantotpercvalue)
    df = df.withColumn("z_scoretotdistabs", pf.abs(df.z_scoretotdist))
    df = df.filter(df.z_scoretotdistabs < 3.5)
    df = df.selectExpr('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')
    return df

#### Task 1.2 - Explanation
I found the median of the trip distance column with percentile approx aggregate function with the percentile value being 0.5 and populated the value in new column 'median'. Collected the calculated value and subtracted it with the trip distance value to get a calculated value and saved it in a column 'mediantotlist'.  I got the absolute value from the mediantotlist column to stored it in a column 'mediantotlistabs'. Then calculated the median of the new column and multiplied it with the constant value 1.438 to get MAD value and stored the value in a variable 'mediantotpercvalue'. Then, I divided the 'mediantotlist' column by 'mediantotpervalue' and populate it a new column 'z_scoretotdist'. I extracted the absolute value of the 'z_scoretotdist' column and stored it in a new column 'z_scoretotdistabs'. Filtered the rows with the value of 'z_scoretotdistabs' greater than 3.5.

I repeated the process again to filter the values from total amount column and selected the necessary columns and overwrote the dataframe to set that as a return value.

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
#display(trips_12)

Row count: 110,871,064

## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

In [0]:
# Your solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    # develop your solution here (create/destroy cells as needed) and then implement it in the functions below
    df = df.join(zones_df, df.PULocationID == zones_df.LocationID).select(df["*"], zones_df["Zone"])
    df = df.withColumnRenamed("Zone", "PickupZone")
    df = df.join(zones_df, df.DOLocationID == zones_df.LocationID)
    df = df.withColumnRenamed("Zone", "DropOffZone")
    df = df.selectExpr('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount', 'PickupZone', 'DropOffZone')
    return df

#### Task 2.1 - Explanation

I joined the trips dataframe with zone_names dataframe with Pickup Location Id from trips dataframe and Location Id from the zone_names dataframes. Second join was between the DropOff Location Id and Location Id. In both joins, I selected the zone column values from zones_names dataframe and renamed as PickupZone and DropOffZone respectively. Then, return the updated dataframe with seven columns.

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
# display(trips_21.take(10))

Row count: 110,871,064

In [0]:
# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    # input: output of task 2.1
    df = df.withColumn("unit_profitability", df.total_amount / df.trip_distance)
    return df

#### Task 2.2 - Explanation

I calculated the unit profitability by dividing the total amount by trip distance and stored in a new column 'unit_profitability'. Then, I returned the dataframe with the updated columns

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
# display(trips_22.take(10))

Row count: 110,871,064

## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
## Your solution to task 3.1 goes HERE
def t31_summarise_trips(df):
    # input: output of task 2.2
    df = df.groupby(df.PULocationID, df.DOLocationID).agg(pf.mean('unit_profitability').alias('average_unit_profit'), 
                                                                   pf.count('trip_distance').alias('trips_count'), pf.sum('passenger_count').alias('total_passengers'))
    return df

#### Task 3.1 - Explanation

With the groupby function, I grouped the rows with the columns 'PULocationID' and 'DOLocationID' to calculate the aggregate mean value of 'unit profitability' column and stored it in a new column 'average_unit_profit'. Then, I calculated the count of trip distance and stored as 'trips_count' column and the sum of the passenger count as 'total_passengers' column. All calculations were made after the groupby function and returned the dataframe.

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
#display(graph.take(10))

Row count: 21,134

In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    df = df.join(zones_df, df.PULocationID == zones_df.LocationID)
    df = df.selectExpr('PULocationID', 'DOLocationID', 'average_unit_profit', 'trips_count', 'total_passengers', 'Zone')
    df = df.withColumnRenamed("Zone", "PickupZone")
    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    zonesreformtrip = df_zones.groupby(df_zones.PickupZone).agg(pf.sum('trips_count').alias('total_trips')) 
    windowvalue = Window.orderBy(pf.col('total_trips').desc())
    output = zonesreformtrip.withColumn('rank', pf.rank().over(windowvalue)).take(10)
    return output

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    zonesreformprofit = df_zones.groupby(df_zones.PickupZone).agg(pf.avg('average_unit_profit').alias('average_profitability'))
    windowvalue = Window.orderBy(pf.col('average_profitability').desc())
    output = zonesreformprofit.withColumn('rank', pf.rank().over(windowvalue)).take(10)
    return output

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    zonesreformpassengers = df_zones.groupby(df_zones.PickupZone).agg(pf.sum('total_passengers').alias('total_passengers'))
    windowvalue = Window.orderBy(pf.col('total_passengers').desc())
    output = zonesreformpassengers.withColumn('rank', pf.rank().over(windowvalue)).take(10)
    return output

#### Task 3.2 - Explanation

The function 'summarise_zones_pairs' joined trips and zone_names dataframe with the common value 'Location Id' with the necessary columns from two dataframes and returned the dataframe for next steps.

The function 't32_top10_trips' got the input from previous function and calculated the sum of the trips count from the groupby function and with the window function, we rearranged the 'total_trips' column into descending order and in the next step, we got the top 10 rank of the zones according to the column 'total_trips'.

The function 't32_top10_profit' calculated the average of the average unit profit from the groupby function and with the window function, we reorder the 'average_profitability' column into descending order and in the next step, we got the top 10 rank of the zones according to the column 'average profitability'.

The function 't32_top10_passenger' calculated the sum of the total passengers travelled from a pickup zone and with the window function, we got the column 'total_passengers' in descending order and in the next step, we got the top 10 rank of the zones according to the column 'total_passengers'.

I used three display functions below for three function to produce a neat output

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_trips)

PickupZone,total_trips,rank
Upper East Side South,5221860,1
Upper East Side North,4787412,2
Midtown Center,4422920,3
Penn Station/Madison Sq West,4001933,4
Midtown East,3904872,5
Lincoln Square East,3479986,6
Murray Hill,3465428,7
Clinton East,3400404,8
Times Sq/Theatre District,3365524,9
Union Sq,3358946,10


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
display(top10_profit)

PickupZone,average_profitability,rank
Newark Airport,774.4899875008736,1
JFK Airport,441.55760916208027,2
Rockaway Park,408.91390583538066,3
Arden Heights,383.54766882398053,4
Far Rockaway,294.1143178738994,5
Ozone Park,249.79557271219295,6
Springfield Gardens South,192.11254163479379,7
Baisley Park,151.7356194966071,8
Rossville/Woodrow,151.5993591035919,9
LaGuardia Airport,141.54721415229676,10


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_passenger)

PickupZone,total_passengers,rank
Upper East Side South,7963691.0,1
Upper East Side North,7309936.0,2
Midtown Center,6905234.0,3
Penn Station/Madison Sq West,6133906.0,4
Midtown East,5995365.0,5
Times Sq/Theatre District,5441559.0,6
Lincoln Square East,5430824.0,7
Clinton East,5353541.0,8
Murray Hill,5315235.0,9
Union Sq,5235143.0,10


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'XXL'
DATA_FORMAT = 'delta'
WITH_TASK_12 = False

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

Trips dataset loaded!
 ---
 Size: XXL
 Format: delta
 Tables loaded: /FileStore/tables/taxi/taxi-XXL-delta/
 Number of trips (dataset rows): 132,396,785

In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

Out[82]: [[Row(PickupZone='Upper East Side South', total_trips=5417225, rank=1),
 Row(PickupZone='Upper East Side North', total_trips=4993025, rank=2),
 Row(PickupZone='Midtown Center', total_trips=4766495, rank=3),
 Row(PickupZone='Penn Station/Madison Sq West', total_trips=4253060, rank=4),
 Row(PickupZone='Midtown East', total_trips=4211261, rank=5),
 Row(PickupZone='Times Sq/Theatre District', total_trips=3759521, rank=6),
 Row(PickupZone='Murray Hill', total_trips=3706378, rank=7),
 Row(PickupZone='Clinton East', total_trips=3684733, rank=8),
 Row(PickupZone='Lincoln Square East', total_trips=3645891, rank=9),
 Row(PickupZone='JFK Airport', total_trips=3628460, rank=10)],
 [Row(PickupZone='Newark Airport', average_profitability=80.03173719148026, rank=1),
 Row(PickupZone='Arden Heights', average_profitability=39.63433605421932, rank=2),
 Row(PickupZone='Rockaway Park', average_profitability=34.65217441493704, rank=3),
 Row(PickupZone='Rossville/Woodrow', average_profitability=23.223031520286007, rank=4),
 Row(PickupZone="Eltingville/Annadale/Prince's Bay", average_profitability=22.449201976606723, rank=5),
 Row(PickupZone='Oakwood', average_profitability=17.641326604768746, rank=6),
 Row(PickupZone='New Dorp/Midland Beach', average_profitability=17.55098659961704, rank=7),
 Row(PickupZone=None, average_profitability=16.589701527446113, rank=8),
 Row(PickupZone='Ozone Park', average_profitability=15.571579160851229, rank=9),
 Row(PickupZone='NV', average_profitability=15.317000109612994, rank=10)],
 [Row(PickupZone='Upper East Side South', total_passengers=8264592.0, rank=1),
 Row(PickupZone='Upper East Side North', total_passengers=7625695.0, rank=2),
 Row(PickupZone='Midtown Center', total_passengers=7456413.0, rank=3),
 Row(PickupZone='Penn Station/Madison Sq West', total_passengers=6523400.0, rank=4),
 Row(PickupZone='Midtown East', total_passengers=6472396.0, rank=5),
 Row(PickupZone='Times Sq/Theatre District', total_passengers=6099950.0, rank=6),
 Row(PickupZone='Clinton East', total_passengers=5815502.0, rank=7),
 Row(PickupZone='JFK Airport', total_passengers=5802482.0, rank=8),
 Row(PickupZone='Lincoln Square East', total_passengers=5694632.0, rank=9),
 Row(PickupZone='Murray Hill', total_passengers=5691254.0, rank=10)]]

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.8 |  15.57 |  41.95 |  90.44 |  132.39 |
| execution time   (w/o 1.2)  | 0.09 | 2.07 | 17.99 | 23.96 | 24.56 |
| execution time              | 0.15 | 3.35 | 33.42 | 45.23 | 48.84 |
| sec / 1M records (w/o 1.2)  | 0.03  | 0.13  | 0.42  | 0.26  | 0.18  |
| sec / 1M records            | 0.05  | 0.22  | 0.79  | 0.57  | 0.36  |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.8 |  15.57 |  41.95 |  90.44 |  132.39 |
| execution time   (w/o 1.2)  | 0.05 | 0.08 | 0.09 | 0.16 | 0.21 |
| execution time              | 0.10 | 0.17 | 0.21 | 0.41 | 0.56 |
| sec / 1M records (w/o 1.2)  | 0.02 | 0.005  | 0.002  | 0.002  | 0.001  |
| sec / 1M records            | 0.03  | 0.01  | 0.005  | 0.004  | 0.004  |